In [2]:
from datasets import get_dataset



In [3]:
import yaml
with open('./configs/config.yaml','r') as f:
    config = yaml.safe_load(f)
ds_config = config['dataset']

dataset = get_dataset(**ds_config)
print(len(dataset))
img, label = dataset[0] 

5994


In [ ]:
from utils.dataloader import get_dataloaders
from utils.get_transforms import get_transforms

train_transform,test_transform = get_transforms()

train_dataset = get_dataset(ds_config['name'],ds_config['root'],'train',train_transform)
test_dataset  = get_dataset(ds_config['name'],ds_config['root'],'test',train_transform)

KeyError: 'root'